In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import lsst.afw.display as afwDisplay
import lsst.afw.image as afwImage

import lsst.afw.coord as afwCoord
import lsst.afw.geom as afwGeom

Define some filters.  This would usually be done for us via the "obs package" (in this case, 
the "subaru" package that knows about SuprimeCam, HSC, and PFS), but in this case we'll do it by hand

In [ ]:
import lsst.afw.image.utils as afwImageUtils

afwImageUtils.resetFilters()
afwImageUtils.defineFilter(name='g', lambdaEff=477, alias=['W-S-G+', 'HSC-G'])
afwImageUtils.defineFilter(name='r', lambdaEff=623, alias=['W-S-R+', 'HSC-R'])
afwImageUtils.defineFilter(name='i', lambdaEff=775, alias=['W-S-I+', 'HSC-I'])
afwImageUtils.defineFilter(name='z', lambdaEff=925, alias=['W-S-Z+', 'HSC-Z'])
afwImageUtils.defineFilter(name='y', lambdaEff=990, alias=['W-S-ZR', 'HSC-Y'])

del afwImageUtils

Read the data image the hard way, piece by piece.  We'll see an easier way in a moment

In [ ]:
fileName = "example1.fits"

md = afwImage.readMetadata(fileName)
wcs = afwImage.makeWcs(md, True)
filter = afwImage.Filter(md)
calib = afwImage.Calib(md)

im = afwImage.ImageF(fileName)
mask = afwImage.MaskU(fileName, 3)
variance = afwImage.ImageF(fileName, 4)

Display the images.  In this case we'll use ds9, but there are other backends available (currently IPAC's firefly, but e.g. ginga would be possible)

In [ ]:
disp = afwDisplay.Display(frame=1)

disp.mtv(im, title="image")
afwDisplay.Display(frame=2).mtv(variance, title="variance")
afwDisplay.Display(frame=3).mtv(mask, title="mask")

Assemble all that data into an "Exposure" -- it has other components too, such as a Psf and knowledge about zeropoints

In [ ]:
mi = afwImage.makeMaskedImage(im, mask, variance)
exp = afwImage.makeExposure(mi)
exp.setCalib(calib)
exp.setFilter(filter)
exp.setWcs(wcs)

Yes, there is an easier way

In [ ]:
exp = afwImage.ExposureF(fileName)
print "%s; lambda_{eff} = %.0f nm" % (exp.getFilter().getName(), filter.getFilterProperty().getLambdaEff())

disp.mtv(exp, title="Exposure")
disp.setMaskTransparency(50)

In [ ]:
if True:
    disp.pan(1670, 265)
else:
    rd = afwCoord.Coord(320.6631635*afwGeom.degrees, -0.3861881*afwGeom.degrees)
    disp.pan(*exp.getWcs().skyToPixel(rd))
    
disp.zoom(2)